## Week five - Lasso Regression
### Assignment Two

<p>First, import necessary libraries. Then load test and train data into them.</p>

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
training = pd.read_csv('kc_house_train_data.csv', dtype = dtype_dict)
testing = pd.read_csv('kc_house_test_data.csv', dtype = dtype_dict)

### Import get_numpy_data from Module Two

In [3]:
def get_numpy_data(data_frame, features, output):
    data_frame['constant'] = 1 # add a constant column to a dataframe
    # prepend variable 'constant' to the features list
    features = ['constant'] + features
    # select the columns of data_frame given by the ‘features’ list into the Frame ‘features_frame’
    features_frame = data_frame[list(features)]
    # this will convert the features_sframe into a numpy matrix
    features_matrix = features_frame.as_matrix()
    # assign the column of data_frame associated with the target to the variable ‘output_array’
    output_column = data_frame[output]
    # this will convert the series into a numpy array:
    output_array = output_column.as_matrix()
    return(features_matrix, output_array)

### And the predict_output function

In [4]:
def predict_output(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

### Write a function to normalize the inputs

In [5]:
def normalize_features(features):
    norms = []
    normalized_features = np.empty(np.shape(features))
    for index in range(features.shape[1]):
        norms.append(np.linalg.norm(features[:,index], axis=0))
        normalized_features[:,index] = features[:,index]/norms[index]
    return (normalized_features, norms)

### Calculate norms of a 2 feature input

In [6]:
sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

In [7]:
np_sales, output_sales = get_numpy_data(sales, ['sqft_living', 'bedrooms'], 'price')


In [8]:
normalized, norms = normalize_features(np_sales)
print np_sales
print normalized

[[  1.00000000e+00   1.18000000e+03   3.00000000e+00]
 [  1.00000000e+00   2.57000000e+03   3.00000000e+00]
 [  1.00000000e+00   7.70000000e+02   2.00000000e+00]
 ..., 
 [  1.00000000e+00   1.02000000e+03   2.00000000e+00]
 [  1.00000000e+00   1.60000000e+03   3.00000000e+00]
 [  1.00000000e+00   1.02000000e+03   2.00000000e+00]]
[[ 0.00680209  0.00353021  0.00583571]
 [ 0.00680209  0.00768869  0.00583571]
 [ 0.00680209  0.00230361  0.00389048]
 ..., 
 [ 0.00680209  0.00305154  0.00389048]
 [ 0.00680209  0.00478673  0.00583571]
 [ 0.00680209  0.00305154  0.00389048]]


### Make a prediction with the initial weights and use that to calculate a step of coordinate descent

In [9]:
initial_weights = np.array([1,4,1])
prediction = predict_output(np_sales, np.transpose(initial_weights))

#Normalize the output
for index in range(len(norms)):
    print 'ro['+str(index)+'] = ' + str(sum(normalized[:,index]*(output_sales - prediction + (initial_weights[index]*normalized[:,index])) ))

ro[0] = 78176568.833
ro[1] = 86601827.9593
ro[2] = 79704172.608


### Calculate Descent for a single feature

In [10]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    normalized, norms = normalize_features(feature_matrix)
    prediction = predict_output(normalized, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = sum(normalized[:,i]*(output - prediction + (weights[i]*normalized[:,i])) )
    print ro_i
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + (l1_penalty/2)
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - (l1_penalty/2)
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [11]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                   [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.475558846691
0.425558846691


### Now define algorithm for cyclical coordinate descent

In [12]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    stopped = False 
    weights = initial_weights
    print range(len(initial_weights))
    while not stopped:
        max_change = 0
        for index in range(len(initial_weights)):
            old_weight = weights[index]
            weights[index] = lasso_coordinate_descent_step(index, feature_matrix, output, weights, l1_penalty)
            change_i = np.abs(old_weight - weights[index])  
            if change_i < tolerance:                
                stopped = True 
    return weights

### Now test

In [13]:
first_weights = [0,0,0]
l1_penalty = 1e7
tolerance = 1.0 
weights = lasso_cyclical_coordinate_descent(np_sales, output_sales, first_weights, l1_penalty, tolerance)

[0, 1, 2]
79400304.6376
15305258.7049
-5299724.16961
70262136.2612
23947595.7648
-4647259.81719
62067326.7428
31161208.2635
-3555758.46421


In [14]:
rss = sum((predict_output(normalized, weights) - output_sales) ** 2)
print rss

2.26732798418e+15


### Now try with more features

In [15]:
features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']

In [16]:
train_cols, train_output = get_numpy_data(training, features, 'price')
test_cols, test_output = get_numpy_data(testing, features, 'price')

In [17]:
train_normalized, train_norms = normalize_features(train_cols)
test_normalized, test_norms = normalize_features(test_cols)

In [18]:
first_weights = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
weights1e7 = lasso_cyclical_coordinate_descent(train_cols, train_output, first_weights, l1_penalty, tolerance)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
71114625.7149
3961380.12528
8743972.47596
10271064.3568
755763.309224
-4138302.98335
12173100.2823
12025131.9563
-10530804.7035
-1352759.50875
5394565.69018
7242690.30918
-7160960.49294
3139465.65362


In [19]:
l1_penalty = 1e8
tolerance = 1.0
weights1e8 = lasso_cyclical_coordinate_descent(train_cols, train_output, first_weights, l1_penalty, tolerance)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
66090269.3443
3164429.92805
10640929.9898
20012015.7683
6285973.79242
11930580.3192
11149738.5115
20983393.1026
5767830.09532
10378834.3663
17829970.6919
17433842.2156
5061885.5376
6918373.46068


In [20]:
l1_penalty = 1e4
tolerance = 5e5
weights1e4 = lasso_cyclical_coordinate_descent(train_cols, train_output, first_weights, l1_penalty, tolerance)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
71114625.7149
3961380.12528
4968442.49245
5356785.21171
-1034778.65448
-8686336.16002
12542789.746
10821991.9822
-8859350.5499
3649520.82307
6544708.94494
2981689.81074
-11585046.4764
3087552.86989
70455350.8481
6437329.79841
8496899.27889
8941642.64001
-1431943.85169
-13549624.1229
7614816.93609
12246226.6784
-12613025.4282
6498184.73376
11824968.6905
3482917.46347
-20026322.5222
3260140.76025
